<a href="https://colab.research.google.com/github/lmassaron/ml4dummies_3ed/blob/main/ML4D3E_17_scoring_opinions_and_sentiments.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
text_1 = "The quick brown fox jumps over the lazy dog."
text_2 = "My dog is quick and can jump over fences."
text_3 = "Your dog is so lazy that it sleeps all the day."
corpus = [text_1, text_2, text_3]

In [2]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer(binary=True, lowercase=False)
vectorizer.fit(corpus)
vectorized_text = vectorizer.transform(corpus)
print(vectorized_text.todense())

[[0 1 0 0 0 1 0 0 1 0 1 0 0 0 1 1 1 1 0 0 0 1]
 [1 0 0 0 1 0 1 0 1 1 0 1 0 1 0 0 1 1 0 0 0 0]
 [0 0 1 1 0 0 0 1 1 0 0 1 1 0 0 1 0 0 1 1 1 1]]


In [3]:
print(vectorizer.vocabulary_)

{'The': 1, 'quick': 17, 'brown': 5, 'fox': 10, 'jumps': 14, 'over': 16, 'the': 21, 'lazy': 15, 'dog': 8, 'My': 0, 'is': 11, 'and': 4, 'can': 6, 'jump': 13, 'fences': 9, 'Your': 2, 'so': 19, 'that': 20, 'it': 12, 'sleeps': 18, 'all': 3, 'day': 7}


In [ ]:
text_4 = "A black dog just passed by but my dog is brown."
corpus.append(text_4)
vectorizer = CountVectorizer()
vectorizer.fit(corpus)
vectorized_text = vectorizer.transform(corpus)
print(vectorized_text.todense()[-1])

[[0 0 1 1 1 1 0 0 2 0 0 1 0 0 0 1 0 1 0 1 0 0 0 0 0 0]]


In [ ]:
from sklearn.feature_extraction.text import TfidfTransformer

tfidf = TfidfTransformer(norm="l1")
tfidf_mtx = tfidf.fit_transform(vectorized_text)

phrase = 3 # choose a number from 0 to 3

total = 0
for word in vectorizer.vocabulary_:
    pos = vectorizer.vocabulary_[word]
    value = list(tfidf_mtx.toarray()[phrase])[pos]
    if value !=0.0:
        print(f"{word:7s}: {value:0.3f}")
        total += value
print('\nSummed values of a phrase: %0.1f' % total)

brown  : 0.095
dog    : 0.126
my     : 0.095
is     : 0.077
black  : 0.121
just   : 0.121
passed : 0.121
by     : 0.121
but    : 0.121

Summed values of a phrase: 1.0


In [ ]:
bigrams = CountVectorizer(ngram_range=(2, 2))
print(bigrams.fit(corpus).vocabulary_)

{'the quick': 30, 'quick brown': 24, 'brown fox': 3, 'fox jumps': 9, 'jumps over': 15, 'over the': 21, 'the lazy': 29, 'lazy dog': 17, 'my dog': 19, 'dog is': 7, 'is quick': 11, 'quick and': 23, 'and can': 1, 'can jump': 6, 'jump over': 14, 'over fences': 20, 'your dog': 31, 'is so': 12, 'so lazy': 26, 'lazy that': 18, 'that it': 27, 'it sleeps': 13, 'sleeps all': 25, 'all the': 0, 'the day': 28, 'black dog': 2, 'dog just': 8, 'just passed': 16, 'passed by': 22, 'by but': 5, 'but my': 4, 'is brown': 10}


In [ ]:
import nltk
nltk.download("punkt_tab")
nltk.download("stopwords")

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

from nltk import word_tokenize
from nltk.stem.porter import PorterStemmer
from nltk.corpus import stopwords

stemmer = PorterStemmer()
stop_words = stopwords.words("english")

def stem_tokens(tokens, stemmer):
    stemmed = []
    for item in tokens:
        stemmed.append(stemmer.stem(item))
    return stemmed

def tokenize(text):
    tokens = word_tokenize(text)
    tokens = [token for token in tokens if token not in stop_words]
    stems = stem_tokens(tokens, stemmer)
    return stems

docs = ["Sam loves swimming so he swims all the time"]
vect = CountVectorizer(tokenizer=tokenize)
vec = vect.fit(docs)

sentence1 = vec.transform(["George loves swimming too! "])

print(vec.get_feature_names_out())
print(sentence1.toarray())

['love' 'sam' 'swim' 'time']
[[1 0 1 0]]


/usr/local/lib/python3.11/dist-packages/sklearn/feature_extraction/text.py:517: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


In [ ]:
import pandas as pd

repository = (
    "https://github.com/lmassaron/ml4dummies_3ed/")
release = "releases/download/v1.0/"
filename = repository + release + "imdb_50k.csv"
reviews = pd.read_csv(filename)

In [ ]:
reviews.sentiment.value_counts()

,count
sentiment,
1,25000
0,25000


In [ ]:
print(reviews.review.sample(1).values[0])

I once lived in the u.p and let me tell you what. I didn't have the foggyest idea what the heck this "bear walk " is. I never heard of it the whole 10 years I was up there. It was really funny in the beginning but went down hill quickly.


In [ ]:
from sklearn.model_selection import train_test_split

train, temp = train_test_split(reviews, test_size=0.4, random_state=0)
valid, test = train_test_split(temp, test_size=0.5, random_state=0)

print(f"Train size: {len(train)}")
print(f"Validation size: {len(valid)}")
print(f"Test size: {len(test)}")

Train size: 30000
Validation size: 10000
Test size: 10000


In [ ]:
import os
os.environ["KERAS_BACKEND"] = "jax"

In [ ]:
import keras

maxlen = 256
vocab_size_limit = 10000

text_vectorization = keras.layers.TextVectorization(
    max_tokens=vocab_size_limit,
    output_mode='int',
    output_sequence_length=maxlen,
    pad_to_max_tokens=True)

text_vectorization.adapt(train.review.values)

def vectorize_text_data(df, vectorizer):
    sequences = vectorizer(df.review.values)
    return sequences, df.sentiment.values

X, y = vectorize_text_data(train, text_vectorization)
Xv, yv = vectorize_text_data(valid, text_vectorization)
Xt, yt = vectorize_text_data(test, text_vectorization)

In [ ]:
keras.utils.set_random_seed(0)

model = keras.models.Sequential()
vocab_size = text_vectorization.vocabulary_size()
embedding_dim = 64

model.add(keras.layers.Input(shape=(maxlen,)))
model.add(keras.layers.Embedding(input_dim=vocab_size,
                                 output_dim=embedding_dim))
model.add(keras.layers.Bidirectional(
    keras.layers.LSTM(32, return_sequences=True)))
model.add(keras.layers.Bidirectional(
    keras.layers.LSTM(32, return_sequences=False)))
model.add(keras.layers.Dropout(0.25))
model.add(keras.layers.Dense(1, activation='sigmoid'))

model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ (None, 256, 64)        │       640,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional (Bidirectional)   │ (None, 256, 64)        │        24,832 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional_1 (Bidirectional) │ (None, 64)             │        24,832 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 1)              │            65 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 689,729 (2.63 MB)

 Trainable params: 689,729 (2.63 MB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
history = model.fit(X, y, epochs=2, batch_size=8,
                    validation_data=(Xv, yv))

Epoch 1/2
3750/3750 ━━━━━━━━━━━━━━━━━━━━ 184s 48ms/step - accuracy: 0.6793 - loss: 0.5808 - val_accuracy: 0.8665 - val_loss: 0.3307
Epoch 2/2
3750/3750 ━━━━━━━━━━━━━━━━━━━━ 179s 47ms/step - accuracy: 0.8774 - loss: 0.3001 - val_accuracy: 0.8756 - val_loss: 0.3009


In [ ]:
from sklearn.metrics import accuracy_score

predictions = (model.predict(Xt) >= 0.5).astype(int)
test_accuracy = accuracy_score(yt, predictions)
print(f"Accuracy on test set: {test_accuracy}")

313/313 ━━━━━━━━━━━━━━━━━━━━ 5s 16ms/step
Accuracy on test set: 0.8792


In [ ]:
from datasets import Dataset
from transformers import AutoTokenizer

model_name = "answerdotai/ModernBERT-base"

tokenizer = AutoTokenizer.from_pretrained(model_name)

def tokenize_function(examples):
    return tokenizer(examples["text"],
                     padding="max_length",
                     truncation=True,
                     max_length=256)

def tokenize_dataset(data):
  data_dict = {'text': data['review'].values, 'labels': data['sentiment'].values}
  dataset = Dataset.from_dict(data_dict)
  return dataset.map(tokenize_function, batched=True)

tokenized_train_dataset = tokenize_dataset(train)
tokenized_valid_dataset = tokenize_dataset(valid)
tokenized_test_dataset = tokenize_dataset(test)

tokenizer_config.json:   0%|          | 0.00/20.8k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.13M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/694 [00:00<?, ?B/s]

Map:   0%|          | 0/30000 [00:00<?, ? examples/s]

Map:   0%|          | 0/10000 [00:00<?, ? examples/s]

Map:   0%|          | 0/10000 [00:00<?, ? examples/s]

In [ ]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained(model_name,
                                                           num_labels=2)

config.json:   0%|          | 0.00/1.19k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/599M [00:00<?, ?B/s]

Some weights of ModernBertForSequenceClassification were not initialized from the model checkpoint at answerdotai/ModernBERT-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
total_params = sum(p.numel() for p in model.parameters())
print(f"Total parameters in {model_name}: {total_params:,}")

Total parameters in answerdotai/ModernBERT-base: 149,606,402


In [ ]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(
    output_dir="./results",
    num_train_epochs=1,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    learning_rate=2e-5,
    weight_decay=0.01,
    report_to="none",
    eval_strategy="steps"
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train_dataset,
    eval_dataset=tokenized_valid_dataset
)

train_result = trainer.train()

/usr/local/lib/python3.11/dist-packages/torch/_inductor/compile_fx.py:194: UserWarning: TensorFloat32 tensor cores for float32 matrix multiplication available but not enabled. Consider setting `torch.set_float32_matmul_precision('high')` for better performance.
  warnings.warn(
W0531 16:48:04.314000 1470 torch/_inductor/utils.py:1137] [1/0] Not enough SMs to use max_autotune_gemm mode


Step,Training Loss,Validation Loss
500,0.363900,0.200232
1000,0.261600,0.216783
1500,0.238800,0.225098
2000,0.226600,0.221852
2500,0.238200,0.197059
3000,0.208600,0.184899
3500,0.181000,0.194398


In [ ]:
import numpy as np
from sklearn.metrics import accuracy_score

predictions = trainer.predict(tokenized_test_dataset)
predicted_labels = np.argmax(predictions.predictions, axis=1)
test_accuracy = accuracy_score(test['sentiment'].values, predicted_labels)
print(f"Accuracy on test set: {test_accuracy}")

Accuracy on test set: 0.9439
